### More Details:

What we wanna do now is to not show the read-later button when we are reading a page which is labeled for later read, instead we want to show a button that can delete a post from the stored_posts list<br>
    
Here is the code for 'PostDetailView' class:

In [ ]:
class PostDetailView(View):
    model = Post
    template_name = "blog/post-detail.html"

    def is_stored_post(self, request, post_id):
        stored_posts = request.session.get('stored_posts')
        if stored_posts is not None:
            is_saved_for_later = post_id in stored_posts
        else:
            is_saved_for_later = False
        return is_saved_for_later  
    def get(self, request, slug):
        post = Post.objects.get(slug = slug)
        return render(request, 'blog/post-detail.html', {
            'post':post,
            'post_tags': post.tags.all(),
            'form':CommentForm(),
            'comments':post.comments.all().order_by("-id"),
            'saved_for_later': self.is_stored_post(request, post.id)
        })

    def post(self, request, slug):
        comment_form = CommentForm(request.POST)
        post = Post.objects.get(slug = slug)
        if comment_form.is_valid():
            comment = comment_form.save(commit=False)
            comment.post = post
            comment.save()
            return HttpResponseRedirect(reverse('blog-post-detail', args=[slug]))
        return render(request, 'blog/post-detail.html', {
            'post':post,
            'post_tags': post.tags.all(),
            'form':comment_form,
            'comments':post.comments.all().order_by("-id"),
            'saved_for_later': self.is_stored_post(request, post.id)
        })

Here is the logic we added to the template:

In [ ]:
<form action="{% url "read-later-url" %}" method="post" class='read-later-form'>
    {% csrf_token %}
    <input type="hidden" name="post_id" value='{{ post.id }}'>
    <button type='submit'>
        {% if saved_for_later %}
            Remove from 'Read-Later' list
        {% else %}
            Read Later
        {% endif %}
    </button>
</form>

The only thing that you want to d is to go to ReadLaterView and remove the post_id if it is already there.<br>

Understand the trick?
<br>
Here is the code:

In [ ]:
def post(self, request):
    stored_posts = request.session.get('stored_posts')
    if stored_posts is None:
        stored_posts = []
    post_id = int(request.POST['post_id'])
    if post_id not in stored_posts:
        stored_posts.append(post_id)
        request.session['stored_posts']=stored_posts
    else:
        stored_posts.remove(post_id)
        request.session['stored_posts']=stored_posts
    return HttpResponseRedirect(reverse('blog-post-page'))

### End of Chapter 11